# **Project Name**    - Netflix Movies and TV Shows clustering



##### **Project Type**    - Unsupervised
##### **Contribution**    - Individual
##### **Student Name -**   Rutik Retwade



# **Project Summary -**

Segmenting Netflix movies and TV shows is an unsupervised machine learning challenge. The initial goal is to prepare the dataset for analysis and clustering.

To begin, I imported the dataset and gathered basic information about it. I then planned the data wrangling and preprocessing steps. One of the initial tasks was converting a date feature, which was originally loaded as an object, into a Pandas datetime object. This conversion makes it easier to work with date-related functions and operations.

Regarding missing values, I chose not to handle them because they were sparse in certain features. These missing values could be useful for exploratory data analysis (EDA) and wouldn't significantly impact the overall analysis.

The next important step was text preprocessing. To prepare the "Description" feature for machine learning, I performed tasks like converting text to lowercase, removing punctuation, stopping words, and eliminating extra white spaces.

Once the text data was preprocessed, I used the TF-IDF vectorization technique to convert the text into numerical format. I set a maximum of 400 features to transform each review observation into a 400-dimensional feature vector.

After obtaining these numerical representations, I moved on to building clustering machine learning models. I experimented with three different models: K-means, Hierarchical Clustering, and DBSCAN algorithms. Each of these methods aimed to group similar movies and TV shows together based on their characteristics.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


**Netflix, the largest online streaming service with over 220 million subscribers, needs to efficiently organize the shows on its platform to improve the user experience and prevent subscriber attrition.**

**Creating clusters of similar shows is essential to better understand and offer personalized show recommendations to consumers based on their preferences.**

**The project's objective is to group Netflix shows into distinct clusters. Shows within the same cluster are similar to one another, while those in different clusters are dissimilar.**

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings


#text preprocessing libraries
import contractions
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer



from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import make_column_transformer


#avoid warnings
warnings.filterwarnings("ignore")

### Dataset Loading

In [ ]:
# Load Dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# LOAD SEOLBIKE DATA SET FROM DRIVE

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Almabetter/ML Unsupervised learning project/NETFLIX MOVIES AND TV SHOWS CLUSTERING.csv')

### Dataset First View

In [ ]:
# Dataset First Look

df.head(5)

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
print("Total Rows in dataset are ",df.shape[0])
print("Total columns in dataset are",df.shape[1])

### Dataset Information

In [ ]:
# Dataset Info
df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
df.duplicated(keep = 'first').sum()

**The observation above indicates that our dataset does not contain any duplicate values.**

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
df.isna().sum()

In [ ]:
# Visualizing the missing values
# Visualizing the missing values
df.isnull().sum().plot(kind= 'bar')
plt.xlabel('Features')
plt.ylabel("Total Number of NaN values")

### What did you know about your dataset?

This is an unsupervised machine learning project, meaning we don't have a target variable. The dataset comprises 12 features and 7787 observations. Initially, it contained some duplicated entries, which I have already removed.

The dataset provides details about movies and series, including various data types such as categorical, text (Description), and numerical data. Additionally, there are four features with missing values: director, cast, country, and date_added.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
df.columns

In [ ]:
# Dataset Describe
df.describe(include='all')

### Variables Description

Here is a description of the dataset's features:

1. **show_id**: A unique identification number for each entry, which may not be particularly useful for analysis.
2. **type**: Describes whether the entry is a series or a movie.
3. **title**: The title of the movie or show.
4. **director**: The name of the director responsible for the production.
5. **cast**: Information about the cast involved in the production.
6. **country**: The country where the movie or series is produced or belongs to.
7. **date_added**: The date when the content was added to Netflix.
8. **release_year**: The year when the movie or series was originally released.
9. **rating**: The TV rating or age rating of the show.
10. **duration**: The duration or length of the movie or series.
11. **listed_in**: Describes the categories or genres to which the content belongs.
12. **description**: Provides a brief description or summary of the movie or series.

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
def unique_counts(df1):
  for i in df.columns.tolist():
    print(f"The Unique Values of Variable ', {i}, 'are:", df[i].unique())
    print()
    print('--'*50)
    print()

unique_counts(df)

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.
#changing date_added feature into pandas datetime

def handle_date_added_feature(date_added_values):
    fin_date = []
    for date in date_added_values:
        if pd.isna(date):
            fin_date.append(np.nan)
        else:
            #extracting day
            day = date.split()[1]
            day = int(day[:-1])
            #extracting month
            month = date.split()[0]
            month_map = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
            month =  month_map[month]
            #extracting year
            year = date.split()[-1]
            fin_date.append(f'{year}-{month}-{day}')
    #returning as datetime
    return pd.to_datetime(fin_date)

In [ ]:
df['date_added'] =  handle_date_added_feature(df.date_added)

In [ ]:
df['cast'] = df['cast'].apply(lambda x : np.nan if pd.isna(x) else x.split(','))

### What all manipulations have you done and insights you found?

Since this is an unsupervised problem, our focus will be on conducting exploratory data analysis (EDA) using most of the available features. Our primary goal is to cluster the data based on the text features.

To streamline our approach, we won't invest significant time in imputing missing values or extensive data wrangling. We've converted the `date_added` feature to a Pandas datetime format for more effective use. Additionally, we've transformed the `listed_in` and `cast` features into lists to facilitate EDA.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code
# Number of Movies and TV Shows in the dataset
plt.figure(figsize=(7,7))
df.type.value_counts().plot(kind='pie',autopct='%1.2f%%')
plt.ylabel('')
plt.title('Movies and TV Shows in the dataset')


##### 1. Why did you pick the specific chart?

A pie chart was chosen to clearly illustrate the dataset's composition by showing the percentage of movies and TV shows. It's a straightforward way to visualize the relative distribution of these categories.

##### 2. What is/are the insight(s) found from the chart?

There are more movies (69.14%) than TV shows (30.86%) in the dataset.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

The insights gained from the dataset can indeed help create a positive business impact for Netflix. Understanding the composition of content (movies and TV shows) allows the platform to refine its content strategy and recommendations, potentially leading to increased user engagement and retention. However, if Netflix were to ignore the preferences of the sizable TV show-watching audience, it might lead to negative growth in that user segment, causing a potential loss in subscribers and revenue. Striking a balance in content offerings remains critical to maintain overall positive business outcomes.

#### Chart - 2

In [ ]:
# Chart - 2 visualization code
# Top 10 directors in the dataset
plt.figure(figsize=(10,5))
df[~(df['director']=='Unknown')].director.value_counts().nlargest(10).plot(kind='barh')
plt.title('Top 10 directors by number of shows directed')

##### 1. Why did you pick the specific chart?

The specific chart chosen, a bar plot, was selected for its effectiveness in visualizing the top 10 directors. Bar plots are particularly straightforward for comparing and interpreting data, making it easy to discern the results and identify the most prominent directors.

##### 2. What is/are the insight(s) found from the chart?

Raul Campos and Jan Suter together have directed 18 movies / TV shows, higher than anyone in the dataset.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

The insight from the chart is that Raul Campos and Jan Suter have directed the most movies/TV shows in the dataset (18). This information can help Netflix collaborate with successful directors for potentially better content. However, if Netflix overfocuses on a few directors, it might limit content diversity and lead to negative growth if users want more variety. Balance is key for positive business impact.

In [ ]:
# Chart - 3 visualization code
tv_show = df[df.type == 'TV Show']
movie = df[df.type == 'Movie']


fig, (ax1, ax2) = plt.subplots(1,2)
tv_show.director.value_counts()[:15].plot(kind='bar',ax = ax1,title='Top 15 TV Show Directors',figsize = (20,8))
movie.director.value_counts()[:15].plot(kind='bar',ax =ax2, title = 'Top 15 Movie Directors',figsize = (20,8))
plt.show()

#### Chart - 3

In [ ]:
# Chart - 3 visualization code
tv_show_cst = []
for obs in tv_show.iterrows():
    if type(obs[1]['cast']) is list:
        tv_show_cst.extend(obs[1]['cast'])

movie_cst = []
for obs in movie.iterrows():
    if type(obs[1]['cast']) is list:
        movie_cst.extend(obs[1]['cast'])


fig, (ax1, ax2) = plt.subplots(1,2)
pd.Series(tv_show_cst).value_counts()[:15].plot(kind='bar',ax = ax1,title='Top 15 TV Show Actors',figsize = (20,8))
pd.Series(movie_cst).value_counts()[:15].plot(kind='bar',ax =ax2, title = 'Top 15 Movie Actors',figsize = (20,8))
plt.show()

del tv_show_cst,movie_cst

##### 1. Why did you pick the specific chart?

I chose bar chart with subplot option in python. I chose bar chart because it helps to understand the data with categorical and numberical feature. Here our categorical data is actors name and on y-axis it is count of their movies/Tvshows in our dataset.

##### 2. What is/are the insight(s) found from the chart?

Takahiro Sakurai acted most TV shows in our dataset with total count of 25 TV Shows. Anupam Kher is the acted most films in our Movie category with the total movie count of 32.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 4

In [ ]:
# Chart - 4 visualization code
# Chart - 3 visualization code

#Checking the distribution of Movie Durations
plt.figure(figsize=(15,15))

plt.subplot(2,2,1)
sns.distplot(movie['duration'].str.extract('(\d+)'),kde=False)
plt.title('Distplot with Normal distribution for Movies',fontweight="bold")

#Checking the distribution of TV SHOWS
plt.subplot(2,1,2)
plt.title("Distribution of TV Shows duration",fontweight='bold')
sns.countplot(x=tv_show['duration'],data=tv_show,order = tv_show['duration'].value_counts().index)
plt.show()

##### 1. Why did you pick the specific chart?

I used histogram to find the distribution between movie and its duration and I used bar-graph to find the distribution between Tv_show and its duration

##### 2. What is/are the insight(s) found from the chart?

Most TV shows in our dataset is released in only one season almost 1600 and Most films released, has 800 minutes duration and it is normaly distributed.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 5

In [ ]:
# Chart - 5 visualization code


# Top 10 Genre in movies
plt.figure(figsize=(18,6))
sns.barplot(x = df["listed_in"].value_counts().head(15).index,
            y = df["listed_in"].value_counts().head(15).values)
plt.xticks(rotation=80)
plt.title("Top10 Genre in Movies",size='16',fontweight="bold")
plt.show()



##### 1. Why did you pick the specific chart?

I selected a bar chart with subplots in Python. The reason for choosing a bar chart is that it's great for presenting data that involves both categorical and numerical features. In this case, our categorical feature is "Genre," and on the y-axis, we represent the count of movies and TV shows. This choice makes it clear and easy to comprehend how content is distributed across different genres in the dataset.

##### 2. What is/are the insight(s) found from the chart?

Genres like Documentaries, Stand-up Comedy, Dramas, and International Movies are highly popular, and they have a significant presence in the Netflix content library. These genres often dominate the platform, indicating a strong offering of these types of content to cater to diverse viewer preferences.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 6

In [ ]:
# Chart - 6 visualization code

# visualizing the movies and tv_shows based on the release year


movies_year =movie['release_year'].value_counts().sort_index(ascending=False)
tvshows_year =tv_show['release_year'].value_counts().sort_index(ascending=False)


sns.set(font_scale=1.4)
movies_year.plot(figsize=(12, 8), linewidth=2.5, label="Movies / year")
tvshows_year.plot(figsize=(12, 8), linewidth=2.5,label="TV Shows / year")
plt.xlabel("Years", labelpad=15)
plt.ylabel("Number", labelpad=15)
plt.title("Production growth yearly", y=1.02, fontsize=22)
plt.legend()
plt.show()

##### 1. Why did you pick the specific chart?

line plot helps us to identify and see the trend clearly especially over time. My aim is to know the count of total number of contents became available over time.

##### 2. What is/are the insight(s) found from the chart?

As we can see that Netflix acheived its peak between 2017 to 2020. This may because of Corona. Because because of corona people stayed in their house which make them to spend more time in internet.So it is clearly understood by Netflix and they make sure to own more contents in that period to attract more subscribers over other OTT platform.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7

In [ ]:
# Chart - 7 visualization code
#Rating based on rating system of all TV Shows

tv_ratings = tv_show.groupby(['rating'])['show_id'].count().reset_index(name='count').sort_values(by='count',ascending=False)
fig_dims = (14,7)
fig, ax = plt.subplots(figsize=fig_dims)
sns.pointplot(x='rating',y='count',data=tv_ratings)
plt.title('TV Show Ratings',size='20')
plt.show()

##### 1. Why did you pick the specific chart?

line plot helps us to find the trend among different category. Here in this we are counting the total of different movies/tv shows ratings.

##### 2. What is/are the insight(s) found from the chart?

TV-MA rating category content is more available in Netflix and followed by TV-14

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 8

In [ ]:
# Chart - 8 visualization code
# Count of TVshow and Movie produced in different country


df_country = df.groupby(['country', 'type'])['show_id'].count().sort_values(ascending = False).reset_index()
plt.figure(figsize = (15, 6))
sns.barplot(data = df_country, x = df_country['country'][:20], y = df_country['show_id'], hue = 'type')
plt.xticks(rotation = 90)
plt.legend(loc = 'upper right')
plt.ylabel('Total Count')
plt.show()

##### 1. Why did you pick the specific chart?

I chose bar chart with subplot option in python. I chose bar chart because it helps to understand the data with categorical and numberical feature. Here our categorical data is Country and on y-axis it is count of TV shows and movies

##### 2. What is/are the insight(s) found from the chart?

As we seen in the word cloud Netflix has more number of United States's Tv shows and Movies. Followed by India, which has highest number of movies and very low number of TV_shows comparing to the others.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 9

In [ ]:
# Chart - 9 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 10

In [ ]:
# Chart - 10 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 11

In [ ]:
# Chart - 11 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 12

In [ ]:
# Chart - 12 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 13

In [ ]:
# Chart - 13 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

HO (Null Hypothesis): Netflix has greater or equal to number of movies collectively.

H1 (Alternate Hypothesis):Netflix has always less number of movies than TV-show collectively.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

from scipy.stats import ttest_ind
import numpy as np

movie_grp = df.groupby(['type',df.date_added.dt.year])['show_id'].count()['Movie']
tv_show = df.groupby(['type',df.date_added.dt.year])['show_id'].count()['TV Show']



movie_grp_mean = np.mean(movie_grp)
tv_show_mean = np.mean(tv_show)

print("Movie Group mean value:",movie_grp_mean)
print("TV_Show mean value:",tv_show_mean)

value_for_mean_std = np.std(movie_grp)
overall_std = np.std(tv_show)

print("Movie std value:",value_for_mean_std)
print("TV_Show std value:",overall_std)

ttest,pval = ttest_ind(movie_grp,tv_show)

print("p-value",pval)
if pval <0.05:
  print("we reject null hypothesis")
else:
  print("we accept null hypothesis")

##### Which statistical test have you done to obtain P-Value?

I did Two-sample T-test.A t-test is a type of inferential statistic which is used to determine if there is a significant difference between the means of two groups which may be related in certain features.

##### Why did you choose the specific statistical test?

 The One Sample t Test determines whether the sample mean is statistically different from a known or hypothesised population mean.
 The One Sample t Test is a parametric test.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

HO:movies rated for kids and older kids are at least two hours long.

H1:movies rated for kids and older kids are not at least two hours long.

#### 2. Perform an appropriate statistical test.

In [ ]:
df_hypothesis = df.copy(deep=True)
df_hypothesis['duration']= df_hypothesis['duration'].str.extract('(\d+)')
df_hypothesis['duration'] = pd.to_numeric(df_hypothesis['duration'])
df_hypothesis['type'] = pd.Categorical(df_hypothesis['type'], categories=['Movie','TV Show'])

#group_by duration and TYPE
group_by_= df_hypothesis[['duration','type']].groupby(by='type')

In [ ]:
# Perform Statistical Test to obtain P-Value
import numpy as np
from scipy import stats


movie_grp = group_by_.get_group('Movie')
tv_show = group_by_.get_group('TV Show')



movie_grp_mean = np.mean(movie_grp)
tv_show_mean = np.mean(tv_show)

print("Movie Group mean value:",movie_grp_mean)
print("TV_Show mean value:",tv_show_mean)

movie_grop_std = np.std(movie_grp)
tv_show_std = np.std(tv_show)

print("Movie std value:",value_for_mean_std)
print("TV_Show std value:",overall_std)


#length of groups and DOF
n1 = len(movie_grp)
n2= len(tv_show)


dof = n1+n2-2

sp_2 = ((n2-1)*movie_grop_std**2  + (n1-1)*tv_show_std**2) / dof

sp = np.sqrt(sp_2)

#tvalue
t_val = (movie_grp_mean-tv_show_mean)/(sp * np.sqrt(1/n1 + 1/n2))
print('tvalue',t_val[0])


if (stats.t.ppf(0.025,dof) < t_val[0]) and (t_val[0] < stats.t.ppf(0.975,dof)):
    print("We Accept the Null Hypothesis")
else :
    print("We reject the Null Hypothesis")

##### Which statistical test have you done to obtain P-Value?

I did Two-sample T-test.A t-test is a type of inferential statistic which is used to determine if there is a significant difference between the means of two groups which may be related in certain features.

##### Why did you choose the specific statistical test?

 The One Sample t Test determines whether the sample mean is statistically different from a known or hypothesised population mean.
 The One Sample t Test is a parametric test.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation

#### What all missing value imputation techniques have you used and why did you use those techniques?


I only require the "Description" feature for training my machine learning model, and since there are no missing values in this feature, there's no need for imputation or handling missing data.







### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments

##### What all outlier treatment techniques have you used and why did you use those techniques?

There is no outliers as I am going use only Description Feature.

### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns

#### What all categorical encoding techniques have you used & why did you use those techniques?

I going to handle only text data here so, I am not going to taking care of other variables here.

### 4. Textual Data Preprocessing
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

#### 1. Expand Contraction

In [ ]:
#decoding non-utf-8 characters
def remove_non_utf8_words(df,features_names):
    df = df.copy()
    for feature in features_names:
        df[feature] = df[feature].apply(lambda x : x.replace('â€™',"'"))
        df[feature] = df[feature].apply(lambda x : ''.join([c for c in x if ord(c) < 128]))
    return df

In [ ]:
# Expand Contraction
def expand_contractions(df,feature_names):
    df = df.copy()
    for feature in feature_names:
        df[feature] = df[feature].apply(lambda x : " ".join(x.split()))
        df[feature] = df[feature].apply(lambda x : contractions.fix(x))
    return df

#### 2. Lower Casing

In [ ]:
# Lower Casing
def change_to_lower_case(df,feature_names):
    df = df.copy()
    for feature in feature_names:
        df[feature] = df[feature].apply(lambda x : x.lower())
    return df

#### 3. Removing Punctuations

In [ ]:
# Remove Punctuations
def remove_punctuations(df,features_columns):
    df = df.copy()
    punctuations = string.punctuation
    for feature in features_columns:
        df[feature] = df[feature].apply(lambda x : x.translate(str.maketrans('','',punctuations)))
    return df

#### 4. Removing URLs & Removing words and digits contain digits.

In [ ]:
# Remove URLs & Remove words and digits contain digits
def remove_urls(df,feature_names):
    df = df.copy()
    for feature in feature_names:
        df[feature] = df[feature].apply(lambda x :  re.sub(r'^https?:\/\/.*[\r\n]*', '', x))
    return df

def remove_words_with_digits(df, feature_names):
    df = df.copy()
    for feature in feature_names:
        df[feature] = df[feature].apply(lambda x : " ".join(s for s in x.split() if not any(c.isdigit() for c in s)))
    return df


#### 5. Removing Stopwords & Removing White spaces

In [ ]:
# Remove Stopwords
def remove_stopwords(df,features_names):
    df = df.copy()
    eng_stopwords = set(stopwords.words('english'))
    for feature in features_names:
        df[feature] = df[feature].apply(lambda text: " ".join(word for word in text.split() if not word in eng_stopwords))
    return df

In [ ]:
# Remove White spaces

#### 6. Rephrase Text

In [ ]:
# Rephrase Text

#### 7. Tokenization

In [ ]:
# Tokenization
def tokenize_and_normalization(df,feature_names):
    lemmatizer=WordNetLemmatizer()
    for feature in feature_names:
            df[feature] = df[feature].apply(lambda x :" ".join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

    return df

#### 8. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)

##### Which text normalization technique have you used and why?

I am using Lemmatization normalization because it will do better than stemming. In stemming, there is chance that it will change the word completely. But in case of Lemmatization, it is not the case, it will try to maintain the original context of the sentence.

#### 9. Part of speech tagging

In [ ]:
# POS Taging

#### 10. Text Vectorization

In [ ]:
class CustomTextDataPreprocessing(BaseEstimator,TransformerMixin):

    def __init__(self,feature_names):
        self.feature_names = feature_names
        return None

    #decoding non-utf-8 characters
    def remove_non_utf8_words(self,df,feature_names):
        df = df.copy()
        for feature in feature_names:
            df[feature] = df[feature].apply(lambda x : x.replace('â€™',"'"))
            df[feature] = df[feature].apply(lambda x : ''.join([c for c in x if ord(c) < 128]))
        return df

    # Expand Contraction
    def expand_contractions(self,df,feature_names):
        df = df.copy()
        for feature in feature_names:
            df[feature] = df[feature].apply(lambda x : " ".join(x.split()))
            df[feature] = df[feature].apply(lambda x : contractions.fix(x))
        return df

        # Lower Casing
    def change_to_lower_case(self,df,feature_names):
        df = df.copy()
        for feature in feature_names:
            df[feature] = df[feature].apply(lambda x : x.lower())
        return df

        # Remove Punctuations
    def remove_punctuations(self,df,features_names):
        df = df.copy()
        punctuations = string.punctuation
        for feature in features_names:
            df[feature] = df[feature].apply(lambda x : x.translate(str.maketrans('','',punctuations)))
        return df

    # Remove URLs & Remove words and digits contain digits
    def remove_urls(self,df,feature_names):
        df = df.copy()
        for feature in feature_names:
            df[feature] = df[feature].apply(lambda x :  re.sub(r'^https?:\/\/.*[\r\n]*', '', x))
        return df

    def remove_words_with_digits(self,df, feature_names):
        df = df.copy()
        for feature in feature_names:
            df[feature] = df[feature].apply(lambda x : " ".join(s for s in x.split() if not any(c.isdigit() for c in s)))
        return df

    # Remove Stopwords
    def remove_stopwords(self,df,feature_names):
        df = df.copy()
        eng_stopwords = set(stopwords.words('english'))
        for feature in feature_names:
            df[feature] = df[feature].apply(lambda text: " ".join(word for word in text.split() if not word in eng_stopwords))
        return df

    # Tokenization
    def tokenize_and_normalization(self,df,feature_names):
        lemmatizer=WordNetLemmatizer()
        for feature in feature_names:
            df[feature] = df[feature].apply(lambda x :" ".join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

        return df


    def fit(self,df):
        return self

    def transform(self,df):
        df =df.copy()
        #removing non utf8 words
        df = self.remove_non_utf8_words(df,self.feature_names)
        #expanding contractions
        df = self.expand_contractions(df,self.feature_names)
        #changing all to lower case
        df = self.change_to_lower_case(df,self.feature_names)
        #remvoing punctuations
        df = self.remove_punctuations(df,self.feature_names)
        #removing urls
        df = self.remove_urls(df,self.feature_names)
        #removing words with digits
        df = self.remove_words_with_digits(df,self.feature_names)
        #remove stopwords
        df = self.remove_stopwords(df,self.feature_names)
        #remove tokenize and normalization
        df = self.tokenize_and_normalization(df,self.feature_names)


        return df

In [ ]:
'''from sklearn.feature_extraction.text import TfidfVectorizer


class TextTransformer:
    def __init__(self, feature_name, max_features=400):
        self.feature_name = feature_name
        self.TfidVectorizer = TfidfVectorizer(max_features=max_features)

    def fit(self, df):
        self.TfidVectorizer.fit(df[self.feature_name])

    def transform(self, df):
        df = df.copy()
        vectors = self.TfidVectorizer.transform(df[self.feature_name]).toarray()
        feature_names = self.TfidVectorizer.get_feature_names_out(input_features=[self.feature_name])
        df = pd.concat([df, pd.DataFrame(vectors, columns=feature_names)], axis=1)
        df.drop(self.feature_name, axis=1, inplace=True)
        return df
'''
# Vectorizing Text
class CustomTfidVectorizer(BaseEstimator,TransformerMixin):
    def __init__(self,feature_name,max_features = None):
        self.max_features = max_features
        self.feature_name = feature_name
        return None

    def fit(self,df):
        self.TfidVectorizer = TfidfVectorizer(max_features= self.max_features)
        self.TfidVectorizer.fit(df[self.feature_name])
        return self

    def transform(self,df):
        df = df.copy()
        vectors = self.TfidVectorizer.transform(df[self.feature_name]).toarray()
        df[self.TfidVectorizer.get_feature_names()] = vectors
        df.drop(self.feature_name,axis = 1,inplace = True)
        return df


##### Which text vectorization technique have you used and why?

Term frequency-inverse document frequency ( TF-IDF) gives a measure that takes the importance of a word into consideration depending on how frequently it occurs in a document and a corpus.Not like Bag of words and Count Vector technique which treats all words equally,TF-IDF can distinguish very common words or rare words

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features

#### 2. Feature Selection

In [ ]:
# Select your features wisely to avoid overfitting

##### What all feature selection methods have you used  and why?

I am going to use only Description feature so I am going to drop rest of the columns.

##### Which all features you found important and why?

I am going to use all the features got from vectorization.

### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data

### 6. Data Scaling

In [ ]:
# Scaling your data

##### Which method have you used to scale you data and why?

There is no need to scale the data I've finally got, I've checked the data, the vectorization got is actually already in normalized form.

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Answer Here.

In [ ]:
# DImensionality Reduction (If needed)

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

I am going to use all the features which I got after vectorization to maintain the information. Dimensionality Reduction doing in the vectorization may loose its meaning of every vector.

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.

##### What data splitting ratio have you used and why?

*No* need to split the data as it is unsupervised problem, we cannot test the result of model by having the test data. so, I am going to use the whole dataset for training.

### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Answer Here.

In [ ]:
# Handling Imbalanced Dataset (If needed)

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Answer Here.

In [ ]:
import nltk
nltk.download('stopwords')




In [ ]:

import nltk
nltk.download('punkt')

In [ ]:
nltk.download('wordnet')


## ***7. ML Model Implementation***

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
class TextTransformer:
    def __init__(self, feature_name, max_features=400):
        self.feature_name = feature_name
        self.TfidVectorizer = TfidfVectorizer(max_features=max_features)

    def fit(self, X, y=None):
        self.TfidVectorizer.fit(X[self.feature_name])
        return self

    def transform(self, X):
        X = X.copy()
        vectors = self.TfidVectorizer.transform(X[self.feature_name]).toarray()
        feature_names = self.TfidVectorizer.get_feature_names_out(input_features=[self.feature_name])
        X = pd.concat([X, pd.DataFrame(vectors, columns=feature_names)], axis=1)
        X.drop(self.feature_name, axis=1, inplace=True)
        return X


# Rest of your code

text_feature_pipeline = Pipeline([
    ('text_preprocessing', CustomTextDataPreprocessing(feature_names=['description'])),
    ('vectorization', TextTransformer(feature_name='description', max_features=400))
])
description_feature_vector = text_feature_pipeline.fit_transform(df)



In [ ]:
#printing the shapes of our data
print("Train data: ",description_feature_vector.shape)

### ML Model - 1

In [ ]:

SEED = 10

In [ ]:
# ML Model - 1 Implementation
#finding optimal number of clusters using the elbow method

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Preprocess and vectorize text data
tfidf_vectorizer = TfidfVectorizer(max_features=400, stop_words='english')
description_feature_vector = tfidf_vectorizer.fit_transform(df['description'])

ssd = []

# Using for loop for iterations from 2 to 14.
for cluster in range(2, 15):
    kmeans = KMeans(n_clusters=cluster, random_state=SEED)
    kmeans.fit(description_feature_vector)
    preds = kmeans.predict(description_feature_vector)
    score = silhouette_score(description_feature_vector, preds)
    print("For n_clusters = {}, Silhouette score is {}".format(cluster, score))
    ssd.append(kmeans.inertia_)




#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

plt.plot(range(2, 15), ssd)
plt.xticks(range(2,15))
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of Squared distance')
plt.show()

From the above Elbow method and silhouette score I am choosing 12 would be the perfect number of clusters for this problem. So I am finally training kmeans with 12 clusters.

In [ ]:
#training the K-means model on a dataset
kmeans = KMeans(n_clusters= 12, init='k-means++', random_state= SEED)
y_predict= kmeans.fit_predict(description_feature_vector)

In [ ]:
#Predict the clusters and evaluate the silhouette score
score = silhouette_score(description_feature_vector, y_predict)
print("Silhouette score is {}".format(score))

In [ ]:
#Reducing the number of features to visualize it in 2D or 3D plot
from sklearn.decomposition import TruncatedSVD

# Reduce the number of features using TruncatedSVD
n_components = 3
svd = TruncatedSVD(n_components=n_components)
X = svd.fit_transform(description_feature_vector)


In [ ]:
plt.figure(figsize=(15,10))
plt.title('customer segmentation based on Recency and Monetary')
plt.scatter(X[:, 0], X[:, 1], c=y_predict, s=50)

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5)
plt.show()

In [ ]:
#plotting 3D Graph
from mpl_toolkits.mplot3d import Axes3D
fig=plt.figure(figsize=(15,10))
plt.title('3d visualization of Recency Frequency and Monetary')
ax=fig.add_subplot(111,projection='3d')
xs=X[:,0]
ys=X[:,1]
zs=X[:,2]
ax.scatter(xs,ys,zs,s=5,c = y_predict)
ax.set_xlabel('PCA 0')
ax.set_ylabel('PCA 1')
ax.set_zlabel('PCA 2')
plt.show()

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
import scipy.cluster.hierarchy as sch

# Convert the sparse matrix to a dense NumPy array
description_feature_array = description_feature_vector.toarray()

plt.figure(figsize=(13, 8))
dendrogram = sch.dendrogram(sch.linkage(description_feature_array, method='ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean Distances')
plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Convert the sparse matrix to a dense NumPy array
description_feature_array = description_feature_vector.toarray()

# Create and fit the AgglomerativeClustering model
aggh = AgglomerativeClustering(n_clusters=9, affinity='euclidean', linkage='ward')
aggh.fit(description_feature_array)

# Predict using the model
y_hc = aggh.fit_predict(description_feature_array)


In [ ]:
#Silhouette Coefficient
print("Silhouette Coefficient: %0.3f"%silhouette_score(description_feature_vector,y_hc, metric='euclidean'))

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
#model 3

#finding optimal number of clusters using the elbow method
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

eps_range=range(6,12) # note, we will scale this down by 100 as we want to explore 0.06 - 0.11 range
minpts_range=range(5,14)

silhouette_scores= []
comb = []

#Using for loop for iterations from 1 to 30.
for k in eps_range:
    for j in minpts_range:
        # Set the model and its parameters+
        model = DBSCAN(eps=k/100, min_samples=j)
        # Fit the model
        clm = model.fit(description_feature_vector)
        # Calculate Silhoutte Score and append to a list
        silhouette_scores.append(silhouette_score(description_feature_vector, clm.labels_, metric='euclidean'))
        comb.append(str(k)+"|"+str(j)) # axis values for the graph

In [ ]:
# Plot the resulting Silhouette scores on a graph
plt.figure(figsize=(20,5))
plt.plot(comb, silhouette_scores, 'bo-')
plt.xlabel('Epsilon/100 | MinPts')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score based on different combnation of Hyperparameters')
plt.show()

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

In [ ]:
final_model = DBSCAN(eps=6/100, min_samples=5)
clm = model.fit(description_feature_vector)
print(silhouette_score(description_feature_vector, clm.labels_, metric='euclidean'))

In [ ]:
plt.scatter(X[:,0], X[:,1], c= clm.labels_)

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Silhouette score helped us to find the best model. And helped us to choose the best model among these.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

K-Means did better job in this dataset. It gives the insight that 12 is the perfect number of cluster, but Incase of Aggloramative Clustering we saw that the 9 is the optimal cluster.Might be 9 is better, but I chose 12 in K-means, I got a better result with that using k-means. And DBSCAN doen't perform well in this dataset.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

filename = 'netflix_agglormative_model.sav'
joblib.dump(aggh, filename)

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**


We had given the problem of unsupervised clustering using Netflix Dataset. Initially we run some basic python code snippets to understand more about the data especially its shape, no.of features, datatypes, statical information etc.. Then in EDA part we analysed the data and found :

Netflix has more number of TV_shows and Movies belongs to United States.

Netflix acheived its peak in owning the number of contents between 2017 to 2020.

Netflix has more number of United States's Tv shows and Movies. Followed by India, which has highest number of movies and very low number of TV_shows comparing to the others.

Clustering:

In K-means which did better, said 12 should be the better cluster.I case of Agglomerative Clustering which too did best job in the Silhoutte score and using the dendogram I found that 9 should be the perfect cluster for this model. But finally using the Silhoutte score I concluded that K-Means did very nice job with 12 clusters.



### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***